Combine preprocessing and indexig

In [1]:
from razdel import tokenize
from nltk.stem.snowball import SnowballStemmer
import string
from nltk.corpus import stopwords

def preprocess_ru(str1, str2 = None, remove_stop: bool=True) -> list:
    """
    Provided text (query text or document title and document text) will be preprocessed by concatenating
    two texts in case of documents, applying tokenization, removing punctuation, stopwords, if indicated, 
    and stemming

    :str1: str, query or document title
    :str2: str, not given or document text
    :param remove_stop: bool indicating if stopwords should be removed (default True)

    :return: list(str) of tokens, stemmed, with removed punctuation
    """
    if str2:
        tokens = [_.text for _ in tokenize(str1 + ' ' + str2)]
    else:
        tokens = [_.text for _ in tokenize(str1)]
    
    preprocessed =[]

    stemmer = SnowballStemmer("russian") 
    stop = stopwords.words("russian")
    punct = string.punctuation + "«»"
    
    for t in tokens:
        if t in punct:
            continue
        if remove_stop and t.lower() in stop:
            continue
        preprocessed.append(stemmer.stem(t))

    return preprocessed 

In [2]:
# doc preprocessing example
title = 'Сергей Аксёнов сообщил о регистрации в Крыму 247 новых случаев COVID-19'
text = 'Глава Республики Крым Сергей Аксёнов сообщил на своей официальной странице в социальной сети «ВКонтакте» о регистрации 247 новых случаев коронавирусной инфекции в Крыму.'

preprocess_ru(title, text, True)

['серг',
 'аксен',
 'сообщ',
 'регистрац',
 'крым',
 '247',
 'нов',
 'случа',
 'COVID-19',
 'глав',
 'республик',
 'крым',
 'серг',
 'аксен',
 'сообщ',
 'сво',
 'официальн',
 'страниц',
 'социальн',
 'сет',
 'вконтакт',
 'регистрац',
 '247',
 'нов',
 'случа',
 'коронавирусн',
 'инфекц',
 'крым']

In [3]:
#query preprocessing example
query = 'Какие некоторые из новейших технологий, которые трансформируют сельскохозяйственную промышленность'
preprocess_ru(query)

['как',
 'некотор',
 'нов',
 'технолог',
 'котор',
 'трансформир',
 'сельскохозяйствен',
 'промышлен']

In [4]:
# Function to build an inverted index
def build_inverted_index(docs):
    inverted_index = {}
    
    for doc in docs:
        # Split content into words
        words = preprocess_ru(doc[1], doc[2])
        
        # Populate the inverted index
        for word in words:
            if word in inverted_index:
                if doc[0] not in inverted_index[word]:
                    inverted_index[word].append(doc[0])
            else:
                inverted_index[word] = [doc[0]]
    
    return inverted_index



In [5]:
#lad datset
import ir_datasets
dataset = ir_datasets.load('neuclir/1/ru/trec-2023')
for doc in dataset.docs_iter()[:10]:
    print(doc)

ExctractedCCDoc(doc_id='ed4af92b-0039-453e-8f25-f359225da8e0', title='Сергей Аксёнов сообщил о регистрации в Крыму 247 новых случаев COVID-19', text='Глава Республики Крым Сергей Аксёнов сообщил на своей официальной странице в социальной сети «ВКонтакте» о регистрации 247 новых случаев коронавирусной инфекции в Крыму.', url='https://mirtesen.ru/dispute/43025555825/Sergey-Aksyonov-soobschil-o-registratsii-v-Kryimu-247-novyih-slu', time=None, cc_file='crawl-data/CC-NEWS/2021/07/CC-NEWS-20210711213016-00623.warc.gz')
ExctractedCCDoc(doc_id='57ab7b0a-eb6e-414a-af2b-dcd9d5faa03f', title='Точно по печени: Тим Цзю простился с Австралией ярким нокаутом', text='Австралийский боксёр российского происхождения Тимофей Цзю победил нокаутом в третьем раунде австралийца Стива Спарка и отстоял свой чемпионский титул WBO Global в первом среднем весе. Теперь сын бывшего абсолютного чемпиона мира намерен продолжить карьеру в Америке или в России и завоевать чемпионский титул в поединках с более серьёзным

In [6]:
#take a sample of 10 first docs
doc_sample = dataset.docs_iter()[:10]
#print(doc_sample)

In [7]:
doc_sample[0][0], doc_sample[0][1]

('ed4af92b-0039-453e-8f25-f359225da8e0',
 'Сергей Аксёнов сообщил о регистрации в Крыму 247 новых случаев COVID-19')

In [8]:
# Construct the inverted index for the toy example documents
inverted_index = build_inverted_index(doc_sample)

# Display the inverted index
print("Inverted Index:")
for word, doc_list in inverted_index.items():
    print(f"{word}: {doc_list}")

Inverted Index:
серг: ['ed4af92b-0039-453e-8f25-f359225da8e0', 'f2d71502-329b-4949-837f-a004d05476f5', '0dfe70a9-c8c8-4524-af58-f010a8bc8e64', 'b18b62a2-18fb-4aa1-adb4-006b3682c2e5']
аксен: ['ed4af92b-0039-453e-8f25-f359225da8e0', 'b18b62a2-18fb-4aa1-adb4-006b3682c2e5']
сообщ: ['ed4af92b-0039-453e-8f25-f359225da8e0', 'f2d71502-329b-4949-837f-a004d05476f5', 'b18b62a2-18fb-4aa1-adb4-006b3682c2e5']
регистрац: ['ed4af92b-0039-453e-8f25-f359225da8e0']
крым: ['ed4af92b-0039-453e-8f25-f359225da8e0', 'b18b62a2-18fb-4aa1-adb4-006b3682c2e5']
247: ['ed4af92b-0039-453e-8f25-f359225da8e0']
нов: ['ed4af92b-0039-453e-8f25-f359225da8e0', '57ab7b0a-eb6e-414a-af2b-dcd9d5faa03f', '16627835-389b-4ad1-bacb-c3c316585197', '64840ae3-bb60-4b73-85b9-9e824476b5b1', '895e41e7-9bcc-472b-9906-678386b55572']
случа: ['ed4af92b-0039-453e-8f25-f359225da8e0', 'b18b62a2-18fb-4aa1-adb4-006b3682c2e5']
COVID-19: ['ed4af92b-0039-453e-8f25-f359225da8e0', 'b18b62a2-18fb-4aa1-adb4-006b3682c2e5']
глав: ['ed4af92b-0039-453e-8f25

In [9]:
#n_collection = len(list(doc_sample))
#print(list(doc_sample))
n_collection = 10

Step-wise implementation of probabilistic retrieval model
1. basic weight - done
2. two Poisson
3. BM-11
4. BM-25

In [10]:
# naive implementation of probabilistic model with basic weight only
import math

def basic_prob_model(query, docs):
    query_terms = set(preprocess_ru(query))
    #print(query_terms)
    
    relevance_scores={}
    

    
    for doc in docs:

        score = 0

        for term in query_terms:
            if term in inverted_index and doc[0] in inverted_index[term]:
                doc_frequency = len(inverted_index[term])
                #print(doc_frequency)
                weight = math.log10(0.5 * n_collection/doc_frequency)
                score+=weight
        relevance_scores[doc[0]] = score
        

    return relevance_scores


In [11]:


for doc in doc_sample:
    doc_terms = list(preprocess_ru(doc[1], doc[2]))
    print(f"Processed terms: {type(doc_terms)}")

In [12]:
# test on toy sample and simple query 
doc_sample = dataset.docs_iter()[:10]
query1 = 'Глава Республики Крым'
basic_prob_model(query1, doc_sample)

{'ed4af92b-0039-453e-8f25-f359225da8e0': 1.3187587626244128,
 '57ab7b0a-eb6e-414a-af2b-dcd9d5faa03f': 0,
 '760feb76-0c3b-40ab-9afc-ff06bc101d41': 0,
 'f2d71502-329b-4949-837f-a004d05476f5': 0,
 'd8cd6f76-9a0c-4997-9cd5-82dfe1df9527': 0,
 '0dfe70a9-c8c8-4524-af58-f010a8bc8e64': 0.2218487496163564,
 '16627835-389b-4ad1-bacb-c3c316585197': 0,
 '64840ae3-bb60-4b73-85b9-9e824476b5b1': 0,
 '895e41e7-9bcc-472b-9906-678386b55572': 0,
 'b18b62a2-18fb-4aa1-adb4-006b3682c2e5': 0.619788758288394}